In [1]:
from model import atp_graph, losses
from data_wrangler import synthetic_data_gen, feature_extractor
import keras
import numpy as np
import pandas as pd
import tensorflow as tf
from model import atp_pipeline
from comparison_models.tnp import tnp_pipeline
from data_wrangler import dataset_preparer
import argparse
from data_wrangler.batcher import batcher, batcher_np
import os
import matplotlib.pyplot as plt
import numpy as np

In [2]:
dataset = "exchange"
model = "atp"
iterations = 300
num_repeats = 1
n_C = 96
n_T = 336

In [3]:


if dataset == "exchange":
    x_train, y_train, x_val, y_val, x_test, y_test = dataset_preparer.dataset_processor(path_to_data="datasets/exchange.csv") 
    save_dir = "weights/forecasting/exchange"
    print('make sure to create the exchange folder in weights/forecasting/')
else: 
    raise ValueError("Dataset not found")
    
save_dir = save_dir + "/" + model

    
batch_size = 32
test_batch_s = 100

nll_list = []
mse_list = []




   



make sure to create the exchange folder in weights/forecasting/


In [4]:
save_dir = "weights/dummy"

In [5]:
x_train, y_train, x_val, y_val, x_test, y_test,_ = dataset_preparer.gp_data_processor(path_to_data_folder="datasets/rbf/") 


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/rbf/x.npy'

# TNP


In [ ]:
i = 0
step = 1
run= 50 + i
tf.random.set_seed(run)


In [ ]:

model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=8,output_shape=48, dropout_rate=0.1, 
                 permutation_repeats=0,bound_std=False, num_layers=6,target_y_dim=1)

Model 1

model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=16,output_shape=32, dropout_rate=0.0, 
                 permutation_repeats=1,bound_std=False, num_layers=6,target_y_dim=1)
                 
Best val score:  0.12        

Model 2 

model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=16,output_shape=32, dropout_rate=0.2, 
                 permutation_repeats=1,bound_std=False, num_layers=6,target_y_dim=1)
                 
Best val score: 0.11



Modle 4
model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=8,output_shape=48, dropout_rate=0.05, 
                 permutation_repeats=1,bound_std=False, num_layers=6,target_y_dim=1)
     
     
0.099




Model 7


model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=12,output_shape=16, dropout_rate=0.1, 
                 permutation_repeats=1,bound_std=False, num_layers=6,target_y_dim=1)
                 
                 
0.22            

Model 8

model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=12,output_shape=16, dropout_rate=0.25, 
                 permutation_repeats=1,bound_std=False, num_layers=6,target_y_dim=1)
                 
0.04    

Model 9 

model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=8,output_shape=48, dropout_rate=0.05, 
                 permutation_repeats=0,bound_std=False, num_layers=6,target_y_dim=1)


-0.6

Model 10 


model = tnp_pipeline.tnp_pipeline(num_heads=6,projection_shape_for_head=12,output_shape=16, dropout_rate=0.15, 
                 permutation_repeats=0,bound_std=False, num_layers=6,target_y_dim=1)
                 
Use model 9

In [ ]:
n_T = 720

In [ ]:
array = []
train_array = []

In [ ]:
idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
x,y,_ = batcher(x_train,y_train,idx_list,window=n_C+n_T,batch_s=32)

In [ ]:
μ, log_σ = model([x, y, n_C, n_T, False])


In [ ]:
μ.shape

In [ ]:
model.summary()

In [ ]:

    run = 54
    tr_step = atp_graph.build_graph()

    ###### can we put the name of the model into the folder name #########?

    name_comp = 'run_' + str(run)
    folder = save_dir + '/ckpt/check_' + name_comp
    if not os.path.exists(folder): os.mkdir(folder)
    opt = tf.keras.optimizers.Adam(3e-4)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
    ckpt.restore(manager.latest_checkpoint) 
    sum_mse_tot = 0; sum_nll_tot = 0
    mini = 50000

In [ ]:
for i in range(2000):
    idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
    _,y,_ = batcher(x_train,y_train,idx_list,window=n_C+n_T) ####### generalise for not just forecasting
    x = np.repeat(np.linspace(-1,1,(n_C+n_T))[np.newaxis,:,np.newaxis],axis=0,repeats=32)
    #### edit batcher to fix this
    _,_, nll_pp_tr, _ = tr_step(model, opt, x,y,n_C,n_T, training=True)

    if i % 100 == 0:
        train_array.append(nll_pp_tr)
        idx_list = list(range(x_val.shape[0] - (n_C+n_T)))
        _,y_te,_ = batcher(x_val,y_val,idx_list,batch_s = 100,window=n_C+n_T)
        t_te = np.repeat(np.linspace(-1,1,(n_C+n_T))[np.newaxis,:,np.newaxis],axis=0,repeats=100)
        μ, log_σ = model([t_te, y_te, n_C, n_T, False])
        _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)
        array.append(nll_pp_te)
        
        plt.plot(train_array)
        plt.plot(array)
        plt.show()

        print(nll_pp_te)
        print(min(array))

        if nll_pp_te < mini:
            mini = nll_pp_te
            manager.save()
            step += 1
            ckpt.step.assign_add(1)




In [ ]:
ckpt = tf.train.Checkpoint(step=tf.Variable(step), optimizer=opt, net=model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint) 

In [ ]:
idx_list = list(range(x_val.shape[0] - (n_C+n_T)))

t_te,y_te,_ = batcher(x_val,y_val,idx_list,batch_s = 100,window=n_C+n_T)
μ, log_σ = model([t_te, y_te, n_C, n_T, False])
_,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)


In [ ]:
nll_pp_te

In [ ]:
plt.plot(y_te[1,:,:])

In [ ]:
sum_mse_tot = 0; sum_nll_tot = 0

idx_list = list(range(x_test.shape[0] - (n_C+n_T)))
num_batches = len(idx_list)//test_batch_s
t_te,y_te,idx_list = batcher(x_test, y_test, idx_list,batch_s = test_batch_s, window=n_C+n_T)
μ, log_σ = model([t_te, y_te, n_C, n_T, False])
_,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)


In [ ]:
nll_pp_te

In [ ]:
msex_te

In [ ]:
test_batch_s

In [ ]:
y_mean = np.mean(y_te[:,:96],axis=1)

In [ ]:
y_mean_repeat = np.repeat(y_mean[:,np.newaxis,:],axis=1,repeats=96)

In [ ]:
np.mean((y_te[:,96:] - y_mean_repeat)**2)

In [ ]:
test_batch_s = 100

sum_mse_tot = 0; sum_nll_tot = 0

idx_list = list(range(x_test.shape[0] - (n_C+n_T)))
num_batches = len(idx_list)//test_batch_s

for _ in range(num_batches): #### specify correct number of batches for the batcher #####
    if(_ == (num_batches-1)): test_batch_s = len(idx_list)        
    _,y_te,idx_list = batcher(x_test, y_test, idx_list,batch_s = test_batch_s, window=n_C+n_T)
    t_te = np.repeat(np.linspace(-1,1,(n_C+n_T))[np.newaxis,:,np.newaxis],axis=0,repeats=y_te.shape[0])
    μ, log_σ = model([t_te, y_te, n_C, n_T, False])
    _, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)
    sum_nll_tot += sum_nll / n_T
    sum_mse_tot += sum_mse / n_T

nllx =  sum_nll_tot / (test_batch_s * x_test.shape[0]//test_batch_s)
msex =  sum_mse_tot / (test_batch_s * x_test.shape[0]//test_batch_s)

print(msex)

In [ ]:
nllx

In [ ]:
msex

In [ ]:
plt.plot(y_te[4])

In [ ]:
idx_list = list(range(x_val.shape[0] - (n_C+n_T)))
t_te,y_te,_ = batcher(x_val,y_val,idx_list,batch_s = 100,window=n_C+n_T)
μ, log_σ = model([t_te, y_te, n_C, n_T, False])
_, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)
print(sum_nll/n_T/100)

In [ ]:
plt.plot(y_te[4])

In [ ]:
plt.plot(y_test[:,0,0])

In [ ]:
plt.plot(y_val[:,0,0])

In [ ]:
sum_nll/n_T/100

In [ ]:
nllx

In [ ]:
msex

In [ ]:
array = np.load("weights/forecasting/exchange/tnp/ckpt/check_run_2/validation_losses_iteration.npy")

In [ ]:
plt.plot(array)

In [ ]:
np.min(array)

In [ ]:
array = np.load("weights/forecasting/exchange/atp/ckpt/check_run_2/validation_losses_iteration.npy")
plt.plot(array)
np.min(array)

In [ ]:
array = np.load("weights/forecasting/exchange/atp/nll_list.npy")
array
array.mean()

In [ ]:
array = np.load("weights/forecasting/exchange/tnp/nll_list.npy")
array
array.mean()

# atp

In [5]:
i = 0
step = 1
run= 50 + i
tf.random.set_seed(run)


Model 1

model = atp_pipeline.atp_pipeline(num_heads=6, projection_shape_for_head=8, output_shape=32, rate=0.1, permutation_repeats=0,
                 bound_std=False, num_layers=3, enc_dim=32, xmin=0.1, xmax=2)
                 
-1.3

Model 2

model = atp_pipeline.atp_pipeline(num_heads=7, projection_shape_for_head=15, output_shape=32, rate=0.1, permutation_repeats=0,
                 bound_std=False, num_layers=3, enc_dim=32, xmin=0.1, xmax=1)
                 
-1.35

Model 3

model = atp_pipeline.atp_pipeline(num_heads=6, projection_shape_for_head=12, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=4, enc_dim=32, xmin=0.1, xmax=1)

-1.35

Model 4 

model = atp_pipeline.atp_pipeline(num_heads=6, projection_shape_for_head=9, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=4, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage=False)
                 
-1.34

Model 5

model = atp_pipeline.atp_pipeline(num_heads=3, projection_shape_for_head=9, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=6, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage=False)
                 
-1.35

Model 6 

model = atp_pipeline.atp_pipeline(num_heads=10, projection_shape_for_head=9, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=6, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage="new_block")

-1.33

Model 7

-1.31

Model 8  

model = atp_pipeline.atp_pipeline(num_heads=8, projection_shape_for_head=12, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=3, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage="xxx")

-1.34

In [6]:
# model = atp_pipeline.atp_pipeline(num_heads=8, projection_shape_for_head=12, output_shape=32, rate=0.0,
#                                   permutation_repeats=0,
#                  bound_std=False, num_layers=3, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage="xxx")



model = atp_pipeline.atp_pipeline(num_heads=10, projection_shape_for_head=9, output_shape=32, rate=0.05, permutation_repeats=0,
                 bound_std=False, num_layers=6, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage="new_block")

array = []
train_array = []

idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
x,y,_ = batcher(x_train,y_train,idx_list,window=n_C+n_T)

μ, log_σ = model([x, y, n_C, n_T, False])


model.summary()

2023-05-09 15:19:16.165197: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-05-09 15:19:16.165755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Max
(32, 336, 432)
(32, 336)
Model: "atp_pipeline"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 feature_wrapper (feature_wr  multiple                 0         
 apper)                                                          
                                                                 
 atp (ATP)                   multiple                  88270     
                                                                 
 de (DE)                     multiple                  4         
                                                                 
Total params: 88,274
Trainable params: 88,272
Non-trainable params: 2
_________________________________________________________________


In [ ]:

    run = 202
    tr_step = atp_graph.build_graph()

    ###### can we put the name of the model into the folder name #########?

    name_comp = 'run_' + str(run)
    folder = save_dir + '/ckpt/check_' + name_comp
    if not os.path.exists(folder): os.mkdir(folder)
    opt = tf.keras.optimizers.Adam(3e-4)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
    ckpt.restore(manager.latest_checkpoint) 
    sum_mse_tot = 0; sum_nll_tot = 0
    mini = 50000

In [ ]:
y.shape

In [ ]:
for i in range(2000):
    idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
    _,y,_ = batcher(x_train,y_train,idx_list,window=n_C+n_T) ####### generalise for not just forecasting
    x = np.repeat(np.linspace(-1,1,(n_C+n_T))[np.newaxis,:,np.newaxis],axis=0,repeats=32)
    #### edit batcher to fix this
    _,_, nll_pp_tr, _ = tr_step(model, opt, x,y,n_C,n_T, training=True)

    if i % 100 == 0:
        train_array.append(nll_pp_tr)
        idx_list = list(range(x_val.shape[0] - (n_C+n_T)))
        _,y_te,_ = batcher(x_val,y_val,idx_list,batch_s = 100,window=n_C+n_T)
        t_te = np.repeat(np.linspace(-1,1,(n_C+n_T))[np.newaxis,:,np.newaxis],axis=0,repeats=100)
        μ, log_σ = model([t_te, y_te, n_C, n_T, False])
        _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)
        array.append(nll_pp_te)
        
        plt.plot(train_array)
        plt.plot(array)
        plt.show()

        print(nll_pp_te)
        print(min(array))

        if nll_pp_te < mini:
            mini = nll_pp_te
            manager.save()
            step += 1
            ckpt.step.assign_add(1)




In [ ]:
ckpt = tf.train.Checkpoint(step=tf.Variable(step), optimizer=opt, net=model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint) 

In [ ]:
test_batch_s = 100

sum_mse_tot = 0; sum_nll_tot = 0

idx_list = list(range(x_test.shape[0] - (n_C+n_T)))
num_batches = len(idx_list)//test_batch_s

for _ in range(num_batches): #### specify correct number of batches for the batcher #####
    if(_ == (num_batches-1)): test_batch_s = len(idx_list)        
    _,y_te,idx_list = batcher(x_test, y_test, idx_list,batch_s = test_batch_s, window=n_C+n_T)
    t_te = np.repeat(np.linspace(-1,1,(n_C+n_T))[np.newaxis,:,np.newaxis],axis=0,repeats=y_te.shape[0])
    μ, log_σ = model([t_te, y_te, n_C, n_T, False])
    _, sum_mse, sum_nll, _, _ = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)
    sum_nll_tot += sum_nll / n_T
    sum_mse_tot += sum_mse / n_T

nllx =  sum_nll_tot / (test_batch_s * x_test.shape[0]//test_batch_s)
msex =  sum_mse_tot / (test_batch_s * x_test.shape[0]//test_batch_s)

print(msex)

## Gp

In [ ]:
i = 0
step = 1
run= 50 + i
tf.random.set_seed(run)


In [ ]:
n_C = 10
n_T = 40

Model 1 

model = atp_pipeline.atp_pipeline(num_heads=3, projection_shape_for_head=10, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=6, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage=True)

-0.61

Model 2

model = atp_pipeline.atp_pipeline(num_heads=3, projection_shape_for_head=9, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=5, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage=False)

-0.7

Model 3 

model = atp_pipeline.atp_pipeline(num_heads=10, projection_shape_for_head=9, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=6, enc_dim=32, xmin=0.1, xmax=2,MHAX_leakage="new_block")
                 
-0.87

Model 4 

model = atp_pipeline.atp_pipeline(num_heads=8, projection_shape_for_head=12, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=3, enc_dim=32, xmin=0.1, xmax=1,MHAX_leakage="xxx")

-1.0

In [ ]:
model = atp_pipeline.atp_pipeline(num_heads=10, projection_shape_for_head=9, output_shape=32, rate=0.0, permutation_repeats=0,
                 bound_std=False, num_layers=6, enc_dim=32, xmin=0.1, xmax=2,MHAX_leakage="new_block")
            
array = []
train_array = []

idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
x,y, = batcher_np(x_train,y_train)

μ, log_σ = model([x, y, n_C, n_T, False])


model.summary()

In [ ]:

    run = 5
    tr_step = atp_graph.build_graph()

    ###### can we put the name of the model into the folder name #########?

    name_comp = 'run_' + str(run)
    folder = save_dir + '/ckpt/check_' + name_comp
    if not os.path.exists(folder): os.mkdir(folder)
    opt = tf.keras.optimizers.Adam(3e-4)
    ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=model)
    manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
    ckpt.restore(manager.latest_checkpoint) 
    sum_mse_tot = 0; sum_nll_tot = 0
    mini = 50000

In [ ]:
for i in range(4000):
    idx_list = list(range(x_train.shape[0] - (n_C+n_T)))
    x,y = batcher_np(x_train,y_train)
    _,_, nll_pp_tr, _ = tr_step(model, opt, x,y,n_C,n_T, training=True)

    if i % 100 == 0:
        train_array.append(nll_pp_tr)
        t_te,y_te = batcher_np(x_val,y_val,batch_s = 100)
        μ, log_σ = model([t_te, y_te, n_C, n_T, False])
        _,_,_, nll_pp_te, msex_te = losses.nll(y_te[:, n_C:n_C+n_T], μ, log_σ)
        array.append(nll_pp_te)
        
        plt.plot(train_array)
        plt.plot(array)
        plt.show()

        print(nll_pp_te)
        print(min(array))

        if nll_pp_te < mini:
            mini = nll_pp_te
            manager.save()
            step += 1
            ckpt.step.assign_add(1)


